#  Code for 2D staggered Lagrangian method in Cartersian coordinates

## Control equations

### Momentum equation

$$\tag{1.1}\rho \frac{d\mathbf{v}}{dt} =  -\nabla p $$ 
where $\mathbf{v}$ is the vector of velocity, and $\rho$ is the density and $p$ is the pressure.  

### Energy equation

$$ \rho \frac{dE}{dt} = -\nabla \cdot (p\mathbf{v})$$ 

where $E$ is the specific total energy and it has a realtion with the specific internal energy 
$$E = e + \frac{1}{2}\mathbf{v}\cdot\mathbf{v}$$

By (1.1) and (1.2) we have 
$$\tag{1.3} \rho \frac{de}{dt} = - p\nabla\cdot \mathbf{v}$$

In  a Cartersian coordinates $x-y$, The equations (1.1) and (1.3) can be written as
$$\rho \frac{du}{dt} = -\frac{\partial p}{\partial x},\quad \rho \frac{dv}{dt} = -\frac{\partial p}{\partial y}$$
and
$$\rho \frac{de}{dt} = -p (\frac{\partial u}{\partial x} +\frac{\partial v}{\partial y})$$

## Compatible Staggered Discretization

<img src="Grid.png" width = "800" height = "400" div align=center /> 

### Quatities:

On point: ($x_p$,$y_p$), ($u_p$,$v_p$)

Zonal:  $V_z$, $\rho_z$, $m_z$, $e_z$, $p_z$

By the relation 
$$\frac{dm_z}{dt} =0$$
we have 
$$\rho_z(t) = m_z/V_z(t)$$

Also 
$$d(x_p)/dt = u_p, \quad d(y_p)/dt = v_p$$

### Point mass 

Point mass $m_p$ is used to solve point momentum $\mu_p = m_p \bf{u}_p$ and kinetic energy $K_p = m_p \frac{\bf{u}_p^2}{2}$

<img src="Ins.jpg" width = "200" height = "400" div align=center /> 

$$m_p = \sum_{z\in Z(p)} A_z^p \rho^p_z$$

$$\rho_z(t) = m_z/V_z(t)$$

In [59]:
function MassAndDens(mz::Array{Float64,1},x::Array{Float64,1},y::Array{Float64,1})
    
    Ic, = size(mz)
    Ip, = size(Tc)
    Asbz = zeros(Float64,4)
    ρ = zeros(Float64,Ic)
    mp = zeros(Float64,Ip)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4)
    for i in 1:Ic
        xz[1:4] = x[Tp[i,1:4]]
        yz[1:4] = y[Tp[i,1:4]]
        Asbz = A_z(i,xz,yz)
        Az = sum(Asbz[j] for j in 1:4)
        ρ[i] = mz[i]/Az
        
        for j = 1:4
            ip = Tp[i,j]
            mp[ip] = mp[ip] + Asbz[j]*ρ[i]
        end
    end
    return ρ,mp
end           
        

MassAndDens (generic function with 3 methods)

In [63]:
mz

4000-element Array{Float64,1}:
 6.250000000000036e-6 
 6.249999999999949e-6 
 6.249999999999948e-6 
 6.25000000000003e-6  
 6.2499999999999884e-6
 6.249999999999881e-6 
 6.250000000000179e-6 
 6.250000000000044e-6 
 6.24999999999999e-6  
 6.2499999999997996e-6
 6.250000000000017e-6 
 6.249999999999691e-6 
 6.250000000000125e-6 
 ⋮                    
 6.250000000006252e-7 
 6.249999999978495e-7 
 6.249999999992373e-7 
 6.250000000061762e-7 
 6.249999999999312e-7 
 6.250000000027069e-7 
 6.249999999971556e-7 
 6.249999999999313e-7 
 6.249999999999311e-7 
 6.249999999992372e-7 
 6.249999999964618e-7 
 6.249999999978495e-7 


$$ A_z^1 = \frac{5A_{41}+5A_{12}+A_{23}+A_{34}}{12}$$
$$ A_z^2 = \frac{A_{41}+5A_{12}+5A_{23}+A_{34}}{12}$$
$$ A_z^3 = \frac{A_{41}+A_{12}+5A_{23}+5A_{34}}{12}$$
$$ A_z^4 = \frac{5A_{41}+A_{12}+A_{23}+5A_{34}}{12}$$

In [15]:
function A_z(ic::Int,x::Array{Float64,1},y::Array{Float64,1}) 
    A =zeros(Float64,4)
    xc= sum(x[i] for i in 1:4)/4
    yc= sum(y[i] for i in 1:4)/4
    
    A41 = Area3(x[1],x[4],xc,y[1],y[4],yc)
    A12 = Area3(x[1],x[2],xc,y[1],y[4],yc)
    A23 = Area3(x[2],x[3],xc,y[2],y[3],yc)
    A34 = Area3(x[3],x[4],xc,y[3],y[4],yc)
    
    A[1] = (5A41+5A12+A23+A34)/12
    A[2] = (A41+5A12+5A23+A34)/12
    A[3] = (A41+A12+5A23+5A34)/12
    A[4] = (5A41+A12+A23+5A34)/12
    
    return A
end

A_z (generic function with 1 method)

<img src="Ins2.jpg" width = "400" height = "300" div align=center />

In [16]:
function F_z(ic::Int,p::Float64)
    for i in 1:4
        x[i],y[i] = Coord(Tp(ic,i))
    end
   
    Fx = zeros(Float64,4)
    Fy = zeros(Float64,4)
    
    L12 = √((x[2]-x[1])^2+(y[2]-y[1])^2)
    n12x = (-y[2]+y[1])/L12
    n12y = (x[2]-x[1])/L12
    
    L41 = √((x[1]-x[4])^2+(y[1]-y[4])^2)
    n41x = (-y[1]+y[4])/L41
    n41y = (x[1]-x[4])/L41
    
    L23 = √((x[3]-x[2])^2+(y[3]-y[2])^2)
    n23x = (-y[3]+y[2])/L23
    n23y = (x[3]-x[2])/L23
    
    L34 = √((x[4]-x[3])^2+(y[4]-y[3])^2)
    n34x = (-y[4]+y[3])/L34
    n34y = (x[4]-x[3])/L34
    
    Fx[1] = p*(L12*n12x+L41*n41x)/2
    Fy[1] = p*(L12*n12y+L41*n41y)/2
    
    Fx[2] = p*(L12*n12x+L23*n23x)/2
    Fy[2] = p*(L12*n12y+L23*n23y)/2
    
    Fx[3] = p*(L23*n23x+L34*n34x)/2
    Fy[3] = p*(L23*n23y+L34*n34y)/2
    
    Fx[4] = p*(L34*n34x+L41*n41x)/2
    Fy[4] = p*(L34*n34y+L41*n41y)/2
    
    return Fx,Fy
    end           

F_z (generic function with 1 method)

对于单元 
$$ m_z \frac{de_z}{dt} = -\sum_{z\in Z(p)} \bf{f}_z^p \cdot \bf{u}_p $$  

对于节点
$$m_p \frac{d\bf{u}_p}{dt} = \sum_{p\in P(z)} \bf{f}_z^p$$

In [17]:
function Rhs(tp::Array{Int,1}=tp, tc=tc,p::Array{Float64,1}=p)  #Right hand e
    Ic, = size(tp)
    Ip, = size(tc)
    rhsu = zeros(Float64,Ip)
    rhsv = zeros(Float64,Ip)
    
    rhse = zeros(Float64,Ic)
    
    for i in 1:Ic
        Fx,Fy = F_z(i,p[i])
        for j in 1:4
            u[j],v[j] = Velocity(Tp(i,j))
        end
        ∑=0
        for j in 1:4
            ∑ = ∑ +u[j]*Fx[j] + v[j]*Fy[j]
        end
        rhze(i) = -∑
        
        for j = 1:4
            ip = Tp(i,j)
            rhsu[ip] = rhsu[ip] + Fx[j]
            rhsv[ip] = rhsv[ip] + Fy[j]
        end     
    end
    return rhse, rhsu,rhsv
end

Rhs (generic function with 4 methods)

### EOS 
$$ e = \frac{p}{(\gamma-1)\rho}$$

In [18]:
function pToe(p,ρ,γ)
    return p ./ ρ/(γ-1)
end
function eTop(e,ρ,γ)
    return (γ-1)*ρ .* e
end

eTop (generic function with 1 method)

In [45]:
function CFL(SF,x::Array{Float64,1},y::Array{Float64,1},
                u::Array{Float64,1},v::Array{Float64},
                p::Array{Float64,1},ρ::Array{Float64,1})
    Ic, = size(ρ)
    cflmin = 1.e9
    cfl =0
    γ = problem.γ
    for i in 1:Ic
        u2max = 0
        u2=0.0
        for j = 1:4
            u2 = u[j]^2+v[j]^2
            if u2 > u2max ; u2max = u2; end
        end
        
        dlmin = 1.e9
        for j=1:3
            dl= (x[j+1] - x[j])^2+(y[j+1]-y[j])^2
            if dl < dlmin; dlmin=dl; end
        end
        c = √(γ*p[i]/ρ[i])
        cfl = √(dlmin)/(√(u2)+c)
        if cfl < cflmin; cflmin = cfl; end
    end
    dt = cfl*SF
    return dt
end 
        

CFL (generic function with 1 method)

In [47]:
function TimeSolve(u,v,x,y,p,mz,ρ)
    tt= problem.tt
    sf =problem.sf
    t= 0.0
    while t<tt
#for i in 1:2
        dt=CFL(sf,x,y,u,v,p,ρ)
        if t+dt>tt
           dt = tt-t
        end
        u,v,x,y,p,ρ = Euler1st(dt,u,v,x,y,p,mz,ρ)
    t += dt
    end
#
    return u,v,x,y,p,ρ
end

TimeSolve (generic function with 1 method)

In [48]:
TimeSolve(u, v, x, y, p, mz,ρ)

UndefVarError: UndefVarError: tc not defined

In [36]:
function Euler1st(dt,u,v,x,y,p,mz)
    Ip, = size(u)
    Ic, = size(p)
    
    ρz,mp = MassAndDens(mz,x,y)
    
    e = pToe(p, ρ,γ)
    rhse, rhsu,rhsv = Rhs(tp, tc, p)
    
    u  += dt*(rhsu ./ mp)
    v  += dt*(rhsv ./ mp)
    e  += dt*(rhse ./ mz)
    
    x  +=  dt*u
    y  +=  dt*v
    
    p = eTop(e, ρ,γ)
    return u,v,x,y,p,ρ
end

Euler1st (generic function with 1 method)

In [88]:
function InitTest1()

    global problem = prb(0.1,0.3,1.4)
     
    I =200
    J =20
    dx = 1.0/I
    dy = 0.1/J
    
    
    Ip = (I+1)*(J+1) # Number of  points
    Ic = I*J  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    IBL = J+1
    IBR = J+1
    IBU = I+1
    IBD = I+1
   # 点格表，格点表初始化 
    for i in 1:200 
        for j in 1:20
            ic = i+(j-1)*200
            Tp[ic,1] = i + (j-1)*201
            Tp[ic,2] = i+1 +(j-1)*201
            Tp[ic,3]=i+1+j*201
            Tp[ic,4]=i+j*201
        end
    end
    
    for i in 1:201
        for j in 1:21
            ip = i+(j-1)*201
            Tc[ip,1] = i +(j-1)*200
            if i==1 
                Tc[ip,2] = 0  
            else
                Tc[ip,2] = i-1+(j-1)*200
            end
            if i==1 || j ==1
                Tc[ip,3] = 0
            else
                Tc[ip,3] = i-1+(j-2)*200
            end
            
            if j==1
                Tc[ip,4] = 0
            else
                Tc[ip,4] = i+(j-2)*200
            end
        end
    end
    
    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    for i in 1:201
        for j = 1:21
            ip =i+(j-1)*201
            x[ip] = i*dx
            y[ip] = j*dy
            
            u[ip] = 0.0
            v[ip] = 0.0
        end
    end
    
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:200
        for j in 1:20
            ic = i+(j-1)*200
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
            A = A_z(ic,xz,yz)
            @show A
            Az[ic] = A[1]+A[2]+A[3]+A[4]
            if i<=100
                mz[ic] = 1.0* Az[ic]
                p[ic] = 1.0
                ρ[ic] = 1.0
            else
                mz[ic] = 0.1 * Az[ic]
                p[ic] = 0.1
                ρ[ic] = 0.1
            end
        end
    end
    
    return x,y,u,v,mz,p,ρ,Az
end

InitTest1 (generic function with 1 method)

In [92]:
x,y,u,v,mz,p,ρ,Az=InitTest1()

A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625

A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625

A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625

A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625

A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625

...

A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625

A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625

A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625

A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625e-6, 2.60417e-6, 4.6875e-6, 5.20833e-7]
A = [-1.5625

([0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05  …  0.96, 0.965, 0.97, 0.975, 0.98, 0.985, 0.99, 0.995, 1.0, 1.005], [0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005  …  0.105, 0.105, 0.105, 0.105, 0.105, 0.105, 0.105, 0.105, 0.105, 0.105], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6  …  6.25e-7, 6.25e-7, 6.25e-7, 6.25e-7, 6.25e-7, 6.25e-7, 6.25e-7, 6.25e-7, 6.25e-7, 6.25e-7], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], [6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6, 6.25e-6  …  6.25e-6, 6.25e-6, 6.2

In [93]:
Az

4000-element Array{Float64,1}:
 6.250000000000036e-6 
 6.249999999999949e-6 
 6.249999999999948e-6 
 6.25000000000003e-6  
 6.2499999999999884e-6
 6.249999999999881e-6 
 6.250000000000179e-6 
 6.250000000000044e-6 
 6.24999999999999e-6  
 6.2499999999997996e-6
 6.250000000000017e-6 
 6.249999999999691e-6 
 6.250000000000125e-6 
 ⋮                    
 6.250000000006251e-6 
 6.249999999978495e-6 
 6.249999999992373e-6 
 6.250000000061762e-6 
 6.249999999999312e-6 
 6.250000000027068e-6 
 6.249999999971555e-6 
 6.2499999999993125e-6
 6.249999999999311e-6 
 6.249999999992372e-6 
 6.249999999964617e-6 
 6.249999999978495e-6 

In [90]:
struct prb
    tt::Float64
    sf::Float64
    γ ::Float64
end

In [15]:
# function Tp(ic::Int,i::Int,tp::Array{Int,1}=tp) #Cell-> Point table
#     return tp[ic,i]
# end
# function Tc(ip::Int, i::Int, tc::Array{Int,1}=tc,tcN=tcN) # Point -> Cell table
#     return tc[ip,i],tcN[ip,i]
# end
function Velocity(ip::Int,u::Array{Float64,1}, v::Array{Float64,1})
    return u[ip],v[ip]
end
function Coord(ip::Int,x::Array{Float64,1}, y::Array{Float64,1})
    return x[ip],y[ip]
end

Coord (generic function with 4 methods)

In [25]:
function Area3(x1,x2,x3,y1,y2,y3)
    return (x1*y2+y1*x3+x2*y3-x1*y3-y1*x2-y2*x3)/2
end

Area3 (generic function with 1 method)

In [31]:
@btime Area3(0,2.2,2.3,0,0,2.4)

  0.013 ns (0 allocations: 0 bytes)


2.64

In [5]:
using BenchmarkTools
@btime sum(i for i in 1:4)

  2.462 ns (0 allocations: 0 bytes)


10

In [31]:
p = ones(Float64,100)
ρ = ones(Float64,100)
e = pToe(p,ρ,1.3)

100-element Array{Float64,1}:
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 ⋮                
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333
 3.333333333333333

In [27]:
global a =3 
function ra(a=a)
    a = 2a
    return a
end
a = ra()
a

6

In [20]:
sum(x[i] for i in 1:4)

0.0

In [2]:
(2,3)

(2, 3)

In [3]:
A= (2,3)

(2, 3)

In [4]:
typeof(A)

Tuple{Int64,Int64}

In [5]:
A

(2, 3)

In [6]:
A[1]

2

In [7]:
A[2]

3

In [8]:
B =[2,3]

2-element Array{Int64,1}:
 2
 3

In [12]:
sum(1,2,3)

MethodError: MethodError: no method matching sum(::Int64, ::Int64, ::Int64)
Closest candidates are:
  sum(::Any, ::Any) at reduce.jl:399
  sum(::Any) at reduce.jl:416
  sum(::Any, !Matched::AbstractArray; dims) at reducedim.jl:649